## Deliverable 2. Create a Customer Travel Destinations Map.

In [6]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
import sys
sys.path.append("..")
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [36]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.tail()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Descriptions
620,620,Hualmay,PE,-11.0964,-77.6139,75.11,72,36,10.33,scattered clouds
621,621,Montepuez,MZ,-13.1256,38.9997,70.41,94,29,2.77,scattered clouds
622,622,Campana,AR,-34.1687,-58.9591,78.91,76,100,7.18,overcast clouds
623,623,Pouembout,NC,-21.1333,164.9,75.0,88,100,4.38,overcast clouds
624,City Name,City,City,City,City,City,City,City,City,City


In [37]:
## Delete last row
city_data_df.drop(labels=[624], axis=0, inplace=True)

In [38]:
city_data_df.tail()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Descriptions
619,619,Shimoda,JP,34.6667,138.95,41.11,87,100,4.83,light rain
620,620,Hualmay,PE,-11.0964,-77.6139,75.11,72,36,10.33,scattered clouds
621,621,Montepuez,MZ,-13.1256,38.9997,70.41,94,29,2.77,scattered clouds
622,622,Campana,AR,-34.1687,-58.9591,78.91,76,100,7.18,overcast clouds
623,623,Pouembout,NC,-21.1333,164.9,75.0,88,100,4.38,overcast clouds


In [39]:
## Changing data types for values
city_data_df = city_data_df.astype({"City_ID": "int64", 
                                    "City": "object",
                                    "Country": "object",
                                    "Lat": "float64", 
                                    "Lng": "float64", 
                                    "Max Temp": "float64", 
                                    "Humidity": "int64", 
                                   "Cloudiness": "int64",
                                    "Wind Speed": "float64"})

city_data_df.dtypes


City_ID                   int64
City                     object
Country                  object
Lat                     float64
Lng                     float64
Max Temp                float64
Humidity                  int64
Cloudiness                int64
Wind Speed              float64
Current Descriptions     object
dtype: object

In [8]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
user_max_temp = float(input("What is your ideal maximum temperature, in degrees Fareneheit, for vacation? "))
user_min_temp = float(input("What is your ideal minimum temperature, in degrees Fareneheit, for vacation? "))


What is your ideal maximum temperature, in degrees Fareneheit, for vacation? 90
What is your ideal minimum temperature, in degrees Fareneheit, for vacation? 60


In [40]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
user_preferred_city_df = city_data_df.loc[(city_data_df["Max Temp"]>=user_min_temp)&(city_data_df["Max Temp"]<=user_max_temp)]
user_preferred_city_df.head(10)


,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Descriptions
1,1,Ouadda,CF,8.0777,22.4007,88.03,23,89,4.23,overcast clouds
2,2,Dharchula,IN,29.8500,80.5333,60.67,32,2,7.27,clear sky
3,3,Hilo,US,19.7297,-155.0900,71.26,93,100,0.00,moderate rain
5,5,Rikitea,PF,-23.1203,-134.9692,79.95,71,70,17.11,broken clouds
6,6,Yulara,AU,-25.2406,130.9889,76.84,44,40,9.22,scattered clouds
9,9,Abha,SA,18.2164,42.5053,64.22,77,0,6.91,clear sky
12,12,Faanui,PF,-16.4833,-151.7500,80.56,73,89,10.83,overcast clouds
13,13,Namibe,AO,-15.1961,12.1522,76.10,82,87,5.66,overcast clouds
17,17,Ouadda,CF,8.0777,22.4007,88.03,23,89,4.23,overcast clouds
18,18,Dharchula,IN,29.8500,80.5333,60.67,32,2,7.27,clear sky


In [42]:
# 4a. Determine if there are any empty rows.
user_preferred_city_df.isnull().sum()

City_ID                 0
City                    0
Country                 3
Lat                     0
Lng                     0
Max Temp                0
Humidity                0
Cloudiness              0
Wind Speed              0
Current Descriptions    0
dtype: int64

In [43]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = user_preferred_city_df.dropna()
clean_df.count()

City_ID                 301
City                    301
Country                 301
Lat                     301
Lng                     301
Max Temp                301
Humidity                301
Cloudiness              301
Wind Speed              301
Current Descriptions    301
dtype: int64

In [45]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Descriptions", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Descriptions,Lat,Lng,Hotel Name
1,Ouadda,CF,88.03,overcast clouds,8.0777,22.4007,
2,Dharchula,IN,60.67,clear sky,29.8500,80.5333,
3,Hilo,US,71.26,moderate rain,19.7297,-155.0900,
5,Rikitea,PF,79.95,broken clouds,-23.1203,-134.9692,
6,Yulara,AU,76.84,scattered clouds,-25.2406,130.9889,
9,Abha,SA,64.22,clear sky,18.2164,42.5053,
12,Faanui,PF,80.56,overcast clouds,-16.4833,-151.7500,
13,Namibe,AO,76.10,overcast clouds,-15.1961,12.1522,
17,Ouadda,CF,88.03,overcast clouds,8.0777,22.4007,
18,Dharchula,IN,60.67,clear sky,29.8500,80.5333,


In [50]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():

    # 6c. Get latitude and longitude from DataFrame
    lat = float(row["Lat"])
    lng = float(row["Lng"]) 
    cty = row["City"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    params["location"] = f"{lat},{lng}"
    hotel_search = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_search["results"][0]["name"]
    except (IndexError):
        print(f"Search: {index} | hotel in {cty} not found.")
        pass
        

Search: 1 | hotel in Ouadda not found.
Search: 17 | hotel in Ouadda not found.
Search: 33 | hotel in Ouadda not found.
Search: 49 | hotel in Ouadda not found.
Search: 102 | hotel in Poum not found.
Search: 154 | hotel in Tanout not found.
Search: 174 | hotel in Taoudenni not found.
Search: 209 | hotel in Mbekenyera not found.
Search: 256 | hotel in Adrar not found.
Search: 273 | hotel in Thinadhoo not found.
Search: 293 | hotel in Poya not found.
Search: 297 | hotel in Marzuq not found.
Search: 314 | hotel in Pangoa not found.
Search: 321 | hotel in Sapa not found.
Search: 329 | hotel in Avera not found.
Search: 394 | hotel in Tadine not found.
Search: 519 | hotel in Mungaa not found.
Search: 578 | hotel in Arteaga not found.


In [69]:
# 7. Drop the rows where there is no Hotel Name.
import numpy as np
hotel_df.loc[hotel_df["Hotel Name"]== "", ["Hotel Name"]] = np.nan
clean_hotel_df = hotel_df.dropna()
clean_hotel_df = clean_hotel_df.reset_index(drop=True)
clean_hotel_df


,City,Country,Max Temp,Current Descriptions,Lat,Lng,Hotel Name
0,Dharchula,IN,60.67,clear sky,29.8500,80.5333,Mahara Hotel
1,Hilo,US,71.26,moderate rain,19.7297,-155.0900,Hilo Hawaiian Hotel
2,Rikitea,PF,79.95,broken clouds,-23.1203,-134.9692,People ThankYou
3,Yulara,AU,76.84,scattered clouds,-25.2406,130.9889,Desert Gardens Hotel - Ayers Rock Resort
4,Abha,SA,64.22,clear sky,18.2164,42.5053,Abha Hotel
...,...,...,...,...,...,...,...
278,Meiganga,CM,80.82,overcast clouds,6.5167,14.3000,DFR HOUSE
279,Hualmay,PE,75.11,scattered clouds,-11.0964,-77.6139,Hotel Villa Kitzia Huacho
280,Montepuez,MZ,70.41,scattered clouds,-13.1256,38.9997,PembaAparthotel
281,Campana,AR,78.91,overcast clouds,-34.1687,-58.9591,Hotel Morada Suites


In [70]:
# 8a. Create the output File (CSV)
output_data_file = ("WeatherPy_vacation.csv")

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [77]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Descriptions} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]


In [78]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.8)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig


Figure(layout=FigureLayout(height='420px'))